# Text 2: Latent semantic indexing
**Internet Analytics - Lab 4**

---

**Group:** *B*

**Names:**

* *Keijiro Tajima*
* *Mahammad Shirinov*
* *Stephen Zhao*

---

#### Instructions

*This is a template for part 2 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [287]:
import pickle
import numpy as np
from scipy.sparse.linalg import svds
from utils import load_json, load_pkl, save_pkl

In [288]:
# Load the original courses data
original_courses = load_json('data/courses.txt')

# To get course data
def get_course_data_by_course_id(course_id):
    for course in original_courses:
        if course['courseId'] == course_id:
            return course
    return None

## Exercise 4.4: Latent semantic indexing

In [289]:
# Load data from previous notebook
TFIDF_matrix = load_pkl('tfidx_matrix.pkl')
col_idx_2_course = load_pkl('courses.pkl')
row_idx_2_term = load_pkl('terms.pkl')

print('Shape of TF-IDF matrix: {}x{}'.format(*TFIDF_matrix.shape))

Shape of TF-IDF matrix: 5466x854


In [290]:
# Perform the SVD
U, Sigma, Vt = svds(TFIDF_matrix, 300)

print('U:     {}x{}'.format(*U.shape))
print('Sigma: {} values'.format(*Sigma.shape))
print('Vt:    {}x{}'.format(*Vt.shape))

U:     5466x300
Sigma: 300 values
Vt:    300x854


### Question 1:  U, V, and S

Firstly, the TF-IDF matrix represents the relationship between terms and documents. Specifically, each of the $14513$ rows is a term represented as its importance/relevance to each of the $854$ courses, and each of the $854$ columns is a course represented as the importances/relevances of the $14513$ terms.

When an SVD is taken with $rank=300$, the matrices $U$ will be $14513\times 300$ and $V^T$ will be $300\times 854$. Now, there is an intermediate matrix $\Sigma$ with dimensions $300\times 300$ which represents the importance of the latent factors / concepts / categories. $\Sigma$ is a diagonal matrix containing the singular values of the original matrix. The $i$th singular value represents the importance of the $i$th latent factor.

Thus, now in $U$, each of the $14513$ rows is a term represented as its "strength of membership" to each of the $300$ latent factors, and each of the $300$ columns is a latent factor represented as the "strengths of membership" of each of the $14513$ terms. In $V^T$, each of the $300$ rows is a latent factor represented as its relevance to each of the $854$ courses, and each of the $854$ columns is a course represented as the relevances of each of the $300$ latent factors.

### Question 2: Top Eigenvalues

In [291]:
top_k = 20

print('Top {} Singular Values and Eigenvalues:'.format(top_k))
print('{:>3} | {:10} | {:10}'.format('top', 'singular', 'eigenvalue'))
for i, s in enumerate(Sigma[-1:-top_k-1:-1]):
    print('{:3d} | {:.8f} | {:.8f}'.format(i+1, s, s**2))

Top 20 Singular Values and Eigenvalues:
top | singular   | eigenvalue
  1 | 3.40303830 | 11.58066966
  2 | 2.88711436 | 8.33542934
  3 | 2.01382221 | 4.05547989
  4 | 1.80711602 | 3.26566831
  5 | 1.47069842 | 2.16295385
  6 | 1.33627799 | 1.78563885
  7 | 1.25738265 | 1.58101113
  8 | 1.22688076 | 1.50523639
  9 | 1.16978520 | 1.36839742
 10 | 1.15404840 | 1.33182772
 11 | 1.14412602 | 1.30902434
 12 | 1.12663846 | 1.26931422
 13 | 1.11318585 | 1.23918274
 14 | 1.11082449 | 1.23393105
 15 | 1.10039847 | 1.21087680
 16 | 1.07791800 | 1.16190721
 17 | 1.04455795 | 1.09110130
 18 | 1.04090002 | 1.08347286
 19 | 1.03812045 | 1.07769406
 20 | 0.99858414 | 0.99717028


## Exercise 4.5: Topic extraction

In [292]:
# Gets the top k (or top all) items in a given list
def get_top_k_idx(arr, k=-1):
    res = [x[0] for x in sorted(enumerate(arr), key=lambda x: x[1], reverse=True)]
    if k == -1:
        return res
    else:
        return res[:k]

### Question 1: Top 10 Topics

In [293]:
num_of_topics = 100
num_of_top_terms = 30
num_of_top_courses = 30

topic_2_top_k_terms = []
topic_2_top_k_courses = []
for topic in range(-1, -num_of_topics-1, -1):
    terms = [(U[idx,topic], row_idx_2_term[idx]) for idx in get_top_k_idx(U[:,topic], num_of_top_terms)]
    courses = [(Vt[topic,idx], col_idx_2_course[idx]) for idx in get_top_k_idx(Vt[topic,:], num_of_top_courses)]
    topic_2_top_k_terms.append(terms)
    topic_2_top_k_courses.append(courses)
    print()
    print('Topic {} (sigma={:.4f}):'.format(-topic, Sigma[topic]))
    print('Terms: {}'.format(', '.join('({:.4f}){}'.format(relevance, term) for relevance, term in terms)))
    print('Courses: {}'.format(', '.join('({:.4f}){}'.format(relevance, course) for relevance, course in courses)))


Topic 1 (sigma=3.4030):
Terms: (-0.0000)1a, (-0.0000)aia, (-0.0000)1b, (-0.0000)incos, (-0.0000)IE, (-0.0000)38, (-0.0000)joel, (-0.0000)1c, (-0.0000)webex, (-0.0000)15288, (-0.0000)na, (-0.0000)crawley, (-0.0000)luenberg, (-0.0000)antithet, (-0.0000)koch, (-0.0000)rudin, (-0.0000)demang, (-0.0000)Va, (-0.0000)criterium, (-0.0000)cvitan, (-0.0000)dixit, (-0.0000)arrow, (-0.0000)mathématiqu, (-0.0000)medina, (-0.0000)hull, (-0.0000)bjork, (-0.0000)braghieri, (-0.0000)pride, (-0.0000)corso, (-0.0000)polic
Courses: (-0.0000)AR-202(c), (-0.0000)COM-414, (-0.0000)AR-401(b), (-0.0001)MATH-457, (-0.0001)AR-402(c), (-0.0001)MATH-438, (-0.0001)MATH-428, (-0.0001)AR-401(c), (-0.0001)BIO-382, (-0.0001)AR-476, (-0.0001)FIN-409, (-0.0001)ME-344, (-0.0001)MATH-483, (-0.0001)MATH-461, (-0.0001)CS-307, (-0.0001)EE-613, (-0.0001)FIN-408, (-0.0001)ME-231(a), (-0.0001)FIN-404, (-0.0001)MATH-469, (-0.0001)AR-402(b), (-0.0001)MATH-641, (-0.0001)CS-212, (-0.0001)CS-470, (-0.0001)MATH-405, (-0.0001)CS-206, 

Terms: (0.2157)plasma, (0.1520)diagnost, (0.1350)fusion, (0.1051)gap, (0.0962)confin, (0.0873)bridg, (0.0857)algebra, (0.0789)optic, (0.0785)programm, (0.0780)linear, (0.0767)theori, (0.0637)experiment, (0.0623)magnet, (0.0582)fiber, (0.0558)data, (0.0548)algorithm, (0.0547)statist, (0.0507)mathematica, (0.0499)laser, (0.0492)hous, (0.0483)edm, (0.0467)nonlinear, (0.0467)experi, (0.0450)modern, (0.0434)space, (0.0404)ring, (0.0392)waveguid, (0.0388)doctor, (0.0374)program, (0.0367)practic
Courses: (0.2917)PHYS-732, (0.1362)PHYS-424, (0.1343)PHYS-731, (0.0993)PHYS-423, (0.0808)PHYS-625, (0.0699)AR-202(c), (0.0633)MATH-726(2), (0.0622)ENG-602, (0.0532)PHYS-445, (0.0530)MATH-726, (0.0508)PHYS-734, (0.0506)MATH-625(2), (0.0506)MATH-625(1), (0.0505)ENG-607, (0.0501)MATH-428, (0.0495)MATH-438, (0.0484)PHYS-608, (0.0476)ENG-601(2), (0.0466)BIO-680, (0.0453)BIO-608, (0.0412)PHYS-449, (0.0377)MATH-625, (0.0376)MATH-641, (0.0375)BIO-449, (0.0356)PHYS-736, (0.0346)BIO-630, (0.0331)CH-332, (0.0330

### Question 2: Labelling the Topics

Now we must label the latent factors... which is not a easy task for the most part. One disadvantage of LSI is the seeming arbitrariness or nonsemanticness of the automatically inferred latent factors. Nonetheless, we will try to attach semantic values to each of the 10 seen above by manually analyzing the courses and terms that make up each of the latent factors.

In [312]:
def get_topic_course_info(i):
    return [(course[0], get_course_data_by_course_id(course[1])) for course in topic_2_top_k_courses[i]]
    
# Print the course descriptions for the ith topic
get_topic_course_info(9)

[(0.47883798423192347,
  {'courseId': 'ENV-525',
   'description': 'This course covers principles of snow physics, snow hydrology, snow-atmosphere interaction and snow modeling. It transmits sound understanding of physical processes within the snow and at its interfaces with the atmosphere and the ground, including field, laboratory, and modeling techniques. Content Processes of snow formation in the atmosphere Physical (thermal, optical, mechanical) properties of snow Snow accumulation, transport, redistribution Heat and mass transfer in snow, metamorphism Energy balance within snow and at its boundaries Processes of snow pack ablation and melt Snow cover variability and interaction with vegetation Snow cover-climate interactions at various scales Measurement methods and field techniques Remote sensing of snow at different scales Approaches of snow cover modeling Snow modeling using the SNOWPACK model Keywords Snow, glaciology, cryosphere, avalanches, hydrology, atmospheric boundary l

In [313]:
topic_2_label = [
    "lack of many terms",
    "Training Rotation",
    "Training Rotation and Field Research Project",
    "Semester Project for IC",
    "Drugs",
    "Optical Fibers",
    "Cellular Biology and Labs",
    "Optics and Lasers",
    "Plasma Physics",
    "Snow",
]

In [314]:
for i, label in enumerate(topic_2_label):
    print("Topic {:2d}: {}".format(i+1, label))

Topic  1: lack of many terms
Topic  2: Training Rotation
Topic  3: Training Rotation and Field Research Project
Topic  4: Semester Project for IC
Topic  5: Drugs
Topic  6: Optical Fibers
Topic  7: Cellular Biology and Labs
Topic  8: Optics and Lasers
Topic  9: Plasma Physics
Topic 10: Snow


## Exercise 4.6: Document similarity search in concept-space

### Search Function

We implement a term-document similarity under LSI `term_course_similarity`. We wrap it in `query_course_similarity` to search for multi-word queries. Finally we wrap it in `search` to return results across the entire corpus.

In [315]:
def term_course_similarity(term_vec, course_vec):
    sigma = np.diag(Sigma)
    res = term_vec.dot(sigma.dot(course_vec)) / (np.linalg.norm(term_vec, 2) * np.linalg.norm(sigma * course_vec, 2))
    return res[0]

def query_course_similarity(query_str, course_id):
    query = np.zeros(Sigma.shape[0])
    query_terms = query_str.strip().split()
    for term in query_terms:
        term_idx = row_idx_2_term.index(term)
        query += U[term_idx,:]
    query /= len(query_terms)
    course_idx = col_idx_2_course.index(course_id)
    course = Vt[:,course_idx].reshape(-1, 1)
    return term_course_similarity(query, course)

def search(query, top_k=5, return_all=False):
    results = [(query_course_similarity(query, course_id), course_id) for course_id in col_idx_2_course]
    results = sorted(results, reverse=True)
    for i in range(top_k):
        course_data = get_course_data_by_course_id(results[i][1])
        print()
        print('== Result {} =='.format(i+1))
        print('{} - {}'.format(course_data['courseId'], course_data['name']))
        print('similarity score: {:.4f}'.format(results[i][0]))
    if return_all:
        return results
    else:
        return results[:top_k]

### Question 1: Doing the Search

In [316]:
markov_chain_results = search('markov chain')


== Result 1 ==
MATH-332 - Applied stochastic processes
similarity score: 4.3349

== Result 2 ==
MGT-484 - Applied probability & stochastic processes
similarity score: 3.0511

== Result 3 ==
EE-605 - Statistical Sequence Processing
similarity score: 2.5041

== Result 4 ==
COM-516 - Markov chains and algorithmic applications
similarity score: 2.4288

== Result 5 ==
MGT-602 - Mathematical models in supply chain management
similarity score: 2.3420


In [317]:
facebook_results = search('facebook')


== Result 1 ==
EE-727 - Computational Social Media
similarity score: 4.8533

== Result 2 ==
EE-593 - Social media
similarity score: 2.8533

== Result 3 ==
HUM-432(a) - How people learn I
similarity score: 1.4028

== Result 4 ==
EE-552 - Media security
similarity score: 1.0880

== Result 5 ==
COM-308 - Internet analytics
similarity score: 1.0733


### Question 2: Comparison with Vector Space Model

The results of this are very pleasing to see. The results for both queries seem to be within reasonable semantic ranges to be deemed correct. Below is a comparison with the results from the previous notebook.

For "markov chains", the "Statistical Sequence Processing" replaces "Supply chain management" to take a place in the top five, and "Markov chains and algorithmic applications" drops one rank.

The similar results indicate that the terms "markov chain" are just the right amount of specificity that introducing a lower-dimensional latent-space does not heavily affect the course descriptions' representations in the term-basis as it crosses the approximative latent-space basis.

For "facebook", whereas the vector space model only found one result, the LSI model was able to pick up enough for a top five listing. The vector space model had only discovered "Computational Social Media", whereas our model was able to pick up more including "Internet Analytics".

The reason LSI finds more courses is because it is not searching for the existance/relevance of the exact term "facebook", but is instead searching for the latent-space approximation of "facebook". This latent-space approximation could be shared with other terms such as "social media". To demonstrate, "social media" is searched below.

In [318]:
social_media_results = search('social media')


== Result 1 ==
EE-727 - Computational Social Media
similarity score: 4.9060

== Result 2 ==
EE-593 - Social media
similarity score: 3.1941

== Result 3 ==
HUM-432(a) - How people learn I
similarity score: 1.6189

== Result 4 ==
COM-308 - Internet analytics
similarity score: 1.2152

== Result 5 ==
EE-552 - Media security
similarity score: 1.1576


## Exercise 4.7: Document-document similarity

### Question 1: Document-Document Similarity Eqn

In [319]:
# We can use cosine similarity in the latent-space to compute the similarity between courses
def course_course_similarity(course_vec_1, course_vec_2):
    return np.dot(course_vec_1, course_vec_2) / (np.linalg.norm(course_vec_1, 2) * np.linalg.norm(course_vec_2, 2))

def search_course(course_id, top_k=5, return_all=False):
    course_idx = col_idx_2_course.index(course_id)
    course = Vt[:,course_idx]
    results = [(course_course_similarity(course, Vt[:,other_idx]), col_idx_2_course[other_idx]) for other_idx in range(len(col_idx_2_course))]
    results = sorted(results, reverse=True)
    for i in range(top_k):
        course_data = get_course_data_by_course_id(results[i][1])
        print()
        print('== Result {} =='.format(i+1))
        print('{} - {}'.format(course_data['courseId'], course_data['name']))
        print('similarity score: {:.4f}'.format(results[i][0]))
    if return_all:
        return results
    else:
        return results[:top_k]
    

### Question 2: Most similar to COM-308

In [320]:
similar_to_com308 = search_course('COM-308')


== Result 1 ==
COM-308 - Internet analytics
similarity score: 1.0000

== Result 2 ==
CS-423 - Distributed information systems
similarity score: 0.5820

== Result 3 ==
CS-401 - Applied data analysis
similarity score: 0.5032

== Result 4 ==
EE-558 - A Network Tour of Data Science
similarity score: 0.4851

== Result 5 ==
EE-727 - Computational Social Media
similarity score: 0.3823


The results of this query are commendable. I often describe *Internet Analytics* as an mathematical introduction to Data Science with focus on applications and it seems like the search results are all within that vein.

**Fun Fact:** One of us is also taking the course "CS-423 - *Distributed Information Systems*" and can definitely attest with first-hand experience that it is **very** similar to *Internet Analytics*, so we are especially happy with the results of this query. DIS goes into detail about search & retrieval, recommender systems, and clustering/classification, leaving out the social-network related content, a significant part of COM-308.